# NLC クロスバリデーションテスト#1

In [1]:
!pip install --upgrade "watson-developer-cloud>=1.3.3"

Requirement already up-to-date: watson-developer-cloud>=1.3.3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: pyOpenSSL>=16.2.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud>=1.3.3)
Requirement not upgraded as not directly required: requests<3.0,>=2.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud>=1.3.3)
Requirement not upgraded as not directly required: python-dateutil>=2.5.3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud>=1.3.3)
Requirement not upgraded as not directly required: Twisted>=13.2.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud>=1.3.3)
Requirement not upgraded as not directly required: autobahn>=0.10.9 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud>=1.3.3)
Requirement not upgraded as not directly requ

In [2]:
# The code was removed by DSX for sharing.

## ICOS(IBM Cloud Object Storage)からファイルを読み込む 

In [3]:
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_57aa90d5d3fb48158eac3f5038e6071e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_1['ENDPOINT'])

body = client_57aa90d5d3fb48158eac3f5038e6071e.get_object(Bucket='bipot1-donotdelete-pr-hjaisgrb6seoj9',Key='abs_select100_con_select100_add_other10.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data = pd.read_csv(body, names=('サンプル文','正解クラス'))
#df_data.head()
df_data.sample(5)

,サンプル文,正解クラス
1137,パラメータをちょっと直しておくかとか,その他10
1097,ただうちの業界経験あるいはある程度そこに,その他9
663,よその会社のそういう統合的な製品を見学に行ってもらったりはしてますけれども,その他5
203,そういうことができますよっていうののセミナーに行ってもらったりはしてますけれどもね,その他1
565,オートバックスはもう全てオートバックス本部がつくったシステムしか採用できないことになってるんで,その他4


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, T_train, T_test = train_test_split(df_data["サンプル文"], df_data["正解クラス"], test_size=0.2)
# X_train 訓練データの入力
# X_test  テストデータの入力
# T_train 訓練データの正解ラベル
# T_test  テストデータの正解ラベル

print('train size:', X_train.shape[0])
print('test size:', X_test.shape[0])

train size: 960
test size: 240


In [5]:
df_train = pd.concat([X_train,T_train],axis=1) #axis=1は横方向の結合を指定します。
df_train.head()

,サンプル文,正解クラス
822,あとオープン系のシステム入れれたらなと考えていまして,その他7
793,どちらにしてももらう形になると思うんで,その他6
748,ほかにももう一つなんですが（？）,その他6
195,それをどうするのかと考えているところがあって,抽象
109,興味があるといえばありますよ,抽象


### トレーニング用データをCSVファイルに保存する

In [6]:
DATA_SET = 'train.csv'
!rm -f $DATA_SET
df_train.to_csv(DATA_SET, sep=",", encoding="utf8", index=False, header=False)
!ls -l $DATA_SET
!head $DATA_SET

-rw-r----- 1 dsxuser dsxuser 80329 May  3 14:18 train.csv
あとオープン系のシステム入れれたらなと考えていまして,その他7
どちらにしてももらう形になると思うんで,その他6
ほかにももう一つなんですが（？）,その他6
それをどうするのかと考えているところがあって,抽象
興味があるといえばありますよ,抽象
そこら辺がユーザー化を意識することなく,その他5
最悪な場合多少Ｗｉｎｄｏｗｓ７もサポート期限切れても使わざるを得ないみたいなね,その他8
要するにその辺がきちんと固まった上での,その他3
それでまたそこにお金つぎ込んどいて,その他4
それは何か有料とか都度都度みたいなそんな感じですかね,その他5


In [7]:
df_test = pd.concat([X_test,T_test],axis=1) #axis=1は横方向の結合を指定します。
df_test.head()

,サンプル文,正解クラス
34,どのPCがどれだけトラフィックを使っているなどの管理も行いたいと思っている,具体
333,どのシステムだけ入れかえるのか,その他2
1002,担当といいましてもヨシナリという者になりまして,その他9
80,グループウェア上にただドキュメントを放り込んでいるだけなので結局どこに何があるか分からない,具体
75,クラウドだったらOffice 365 GoogleAppsあたりは話としてあがってくる,具体


In [8]:
# The code was removed by DSX for sharing.

In [9]:
from watson_developer_cloud import NaturalLanguageClassifierV1

natural_language_classifier = NaturalLanguageClassifierV1(
    username = credentials_nlc['username'],
    password = credentials_nlc['password'])

In [10]:
natural_language_classifier.set_default_headers({'x-watson-learning-opt-out': "true"})

In [11]:
def create_classifier(mode):
    # fetch all classifiers associated with the NLC instance
    result = natural_language_classifier.list_classifiers()
    # for the purposes of this demo, we handle only one classifier
    # return the first one found
    if mode == "list":
        if len(result['classifiers']) > 0:
            return result['classifiers']
        else:
            return {"error": "クラスを作成してください"}
    if mode == "add":
        print ('creating class ...')
        # if none found, create a new classifier, change this value
        with open(DATA_SET, 'rb') as training_data:
            metadata = '{"name": "abs_conc_class", "language": "ja"}'
            classifier = natural_language_classifier.create_classifier(
                metadata=metadata,
                training_data=training_data
            )
        return classifier

In [17]:
create_classifier('list')

{'error': 'クラスを作成してください'}

### クラスの作成

In [18]:
myclass = create_classifier('add')
myclass

creating class ...


{'classifier_id': 'ea433fx368-nlc-917',
 'created': '2018-05-03T14:20:17.147Z',
 'language': 'ja',
 'name': 'abs_conc_class',
 'status': 'Training',
 'status_description': 'The classifier instance is in its training phase, not yet ready to accept classify requests',
 'url': 'https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/ea433fx368-nlc-917'}

### クラスが使える状態かのチェック

In [19]:
from time import sleep
from datetime import datetime, timezone, timedelta

JST = timezone(timedelta(hours=+9), 'JST')
#CID='e998c1x363-nlc-976' #上記 classifier_idをセットする
CID=myclass["classifier_id"]

while True:
    status = natural_language_classifier.get_classifier(CID)
    if status["status"] == "Available":
        print ('できたよ!!! 次に進め ...')
        break
    else:
        print ('まだだよ... ', datetime.now(JST).strftime("%Y/%m/%d %H:%M:%S"))
        sleep(60)

まだだよ...  2018/05/03 23:20:23
まだだよ...  2018/05/03 23:21:30
まだだよ...  2018/05/03 23:22:30
まだだよ...  2018/05/03 23:23:31
まだだよ...  2018/05/03 23:24:31
まだだよ...  2018/05/03 23:25:31
まだだよ...  2018/05/03 23:26:31
まだだよ...  2018/05/03 23:27:31
まだだよ...  2018/05/03 23:28:32
まだだよ...  2018/05/03 23:29:32
まだだよ...  2018/05/03 23:30:32
まだだよ...  2018/05/03 23:31:32
まだだよ...  2018/05/03 23:32:33
まだだよ...  2018/05/03 23:33:33
まだだよ...  2018/05/03 23:34:33
できたよ!!! 次に進め ...


### テストデータでクラスを評価する

In [ ]:
#テストデータを使ってクラスを評価する
ok=0
ng=0
#for column_name, item in df_test.sample(30).iterrows():
#for column_name, item in df_test.head().iterrows():
print ('START... ', datetime.now(JST).strftime("%Y/%m/%d %H:%M:%S"))
for column_name, item in df_test.iterrows():
    #target = item[0][0:13] #先頭から15文字分取得
    target = item[0] # item[0]:1列目(サンプル文)を取得する
    classes = natural_language_classifier.classify(CID, target)
    if classes["classes"][0]["class_name"][:3] == item[1][:3]:
        ok += 1
    else:
        ng += 1
    print('[%5s] %4s(%0.2f)' % (classes["classes"][0]["class_name"][:3] == item[1][:3], # item[1]:2列目の最初の3文字 (教師データ)
                                classes["classes"][0]["class_name"], 
                                classes["classes"][0]["confidence"]), end="")
    
    print(' << (%s) %s >>' % (item[1], target))

print ('トータル = %d, OK = %d, NG = %d' % ((ok+ng),ok,ng))
print ('正解率 = %0.2f' % (ok/(ok+ng)))
print ('エラー率 = %0.2f' % (ng/(ok+ng)))
print ('END... ', datetime.now(JST).strftime("%Y/%m/%d %H:%M:%S"))

START...  2018/05/03 23:36:38
[ True]   具体(0.98) << (具体) どのPCがどれだけトラフィックを使っているなどの管理も行いたいと思っている >>
[ True] その他10(0.88) << (その他2) どのシステムだけ入れかえるのか >>
[ True] その他4(0.24) << (その他9) 担当といいましてもヨシナリという者になりまして >>
[ True]   具体(0.99) << (具体) グループウェア上にただドキュメントを放り込んでいるだけなので結局どこに何があるか分からない >>
[ True]   具体(0.97) << (具体) クラウドだったらOffice 365 GoogleAppsあたりは話としてあがってくる >>
[ True] その他3(0.93) << (その他1) リアルタイムでとかっていうことを非常におっしゃられるんですけど >>
[ True]   抽象(0.73) << (抽象) ちょっと予算が狂った >>
[ True] その他9(0.35) << (その他5) あとその音声からテキスト起こしてそれをＡＩでくってくとかですね >>
[ True] その他8(0.48) << (その他5) どこまで何をやってくれるかによると思うんですよ >>
[ True] その他5(0.86) << (その他2) 普段使っていない中旬とか月の途中とかっていうときには >>
[ True] その他3(0.82) << (その他3) もう一応来期の予算ってもうほぼ固まってきてる段階なんで >>
[ True] その他5(0.44) << (その他5) この人一人っていうのがないので >>
[ True] その他3(0.50) << (その他6) 監視とかどういうふうにしてくれるのかっていうのは >>
[ True] その他10(0.90) << (その他10) メールですとかまたお電話ですとかで＊＊＊ >>
[ True] その他10(0.49) << (その他2) １６が出るちょうど前ぐらいだったんで >>
[ True] その他1(0.65) << (その他4) そういう意味ですとまだ実際にどこのどちらの >>
[ True] その他3(0.76) << (その他5) マイクロソフト３

# (必要な場合) クラスの削除

In [16]:
create_classifier('list')

{'error': 'クラスを作成してください'}

In [15]:
DELCID='e9fa7bx367-nlc-898' #上記 classifier_idをセットする
natural_language_classifier.delete_classifier(DELCID)

# バックアップ

In [ ]:
#これもうまくいったが...

for column_name, item in df_data.iloc[:,0].sample(10).iteritems():
    target = item[0:15] #先頭から15文字分取得
    classes = natural_language_classifier.classify(CLASS, target)
    print('%4s(%f)' % (classes["classes"][0]["class_name"], classes["classes"][0]["confidence"]), end="")
    print(' << %s >>' % target)

In [ ]:
#トレーニング用データフレームの作成
df_train = pd.DataFrame(X_train)
df_train['回答クラス'] = T_train
df_train.head()

In [ ]:
#テスト用データフレームの作成
df_test = pd.DataFrame(X_test)
df_test['回答クラス'] = T_test
df_test.head()

# データフレーム操作

In [ ]:
df_data.index

In [ ]:
df_data.columns

In [ ]:
df_data.shape

In [ ]:
df_data.info()

In [ ]:
df_data["サンプル文"].head()

In [ ]:
df_data.loc[:,"サンプル文"].head()

In [ ]:
df_data.iloc[:,0].head()